In [1]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
nasa <- as_data_frame(nasa)

Warning message:
“`as_data_frame()` is deprecated, use `as_tibble()` (but mind the new semantics).
This warning is displayed once per session.”


# Data partition

In [3]:
mechkar::train_test(data=nasa,train_name="train",test_name="test",prop=0.7,seed=1,tableone=TRUE)

Dataset partitioned into:

 + Train dataset: train

 + Test dataset: test



  |======================================================================| 100%

Coefficient covariances computed by hccm()

You got a perfectly balanced training and test datasets

 



V1,V2,Pop,1,2,pval
<chr>,<chr>,<chr>,<fct>,<fct>,<fct>
Individuals,n,41472,29030,12442,NA
lat,Mean (SD),7.5 (17.3),7.5 (17.3),7.6 (17.3),NA
lat,Median (IQR),7.5 (-6.9-21.9),6.3 (-8.7-21.2),8.7 (-6.2-23.7),0.39
long,Mean (SD),-85.0 (17.3),-85.0 (17.3),-85.0 (17.3),NA
long,Median (IQR),-85.0 (-99.4--70.6),-86.3 (-101.3--68.7),-83.7 (-98.8--71.2),0.989
month,Mean (SD),6.5 (3.5),6.5 (3.5),6.5 (3.4),NA
month,Median (IQR),6.5 (3.8-9.2),6.0 (3.0-9.0),7.0 (4.0-10.0),0.159
year,Mean (SD),"1,997.5 (1.7)","1,997.5 (1.7)","1,997.5 (1.7)",NA
year,Median (IQR),"1,997.5 (1,996.0-1,999.0)","1,997.0 (1,996.0-1,999.0)","1,998.0 (1,996.0-1,999.0)",0.836


# Models

In [4]:
### The error we will use is the RMSE(Root Mean Square Error) and RMSLE (Root Mean Square Logaritmic Error)
rmse <- function(y,y_hat) {
    err <- sqrt(sum((y_hat-y)^2,na.rm=T)/length(y))
    return(err)
}

rmsle <- function(y,y_hat) {
    err <- sqrt(sum((log(y_hat+1)-log(y+1))^2,na.rm=T)/length(y))
    return(err)
}

In [5]:
### Table of resulting errors
### Name, Model, RMSE, RMSLE
err_res <- NULL

# Linear Models

In [7]:
## model with only the original variables
mod1 <- lm(temperature ~., data=train)
summary(mod1)


Call:
lm(formula = temperature ~ ., data = train)

Residuals:
     Min       1Q   Median       3Q      Max 
-14.4641  -1.1324   0.0089   1.1756  11.6171 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -4.162e+02  1.322e+01 -31.486  < 2e-16 ***
lat         -3.613e-02  9.212e-04 -39.221  < 2e-16 ***
long         2.978e-03  7.022e-04   4.240 2.24e-05 ***
month        1.133e-01  3.254e-03  34.818  < 2e-16 ***
year         2.446e-01  6.629e-03  36.898  < 2e-16 ***
cloudhigh    2.854e-02  1.320e-03  21.617  < 2e-16 ***
cloudlow    -8.341e-02  1.081e-03 -77.187  < 2e-16 ***
cloudmid     5.904e-03  1.696e-03   3.482 0.000498 ***
ozone       -2.693e-02  8.153e-04 -33.026  < 2e-16 ***
pressure     3.386e-02  3.129e-04 108.225  < 2e-16 ***
surftemp     6.783e-01  3.011e-03 225.229  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.897 on 28947 degrees of freedom
  (72 observations deleted due to missingne

In [12]:
pred1a <- predict(mod1,newdata=test)
rmse(test$temperature,pred1a)
rmsle(test$temperature,pred1a)
err_res <- rbind(err_res, data.frame(Name="Base Linear regression", Model="mod1", 
                                     RMSE=rmse(test$temperature,pred1a), 
                                     RMSLE=rmsle(test$temperature,pred1a)))

[1] 1.876241

[1] 0.006330632

In [13]:
## model with all the variables
mod2 <- lm(temperature ~., data=train)
summary(mod2)


Call:
lm(formula = temperature ~ ., data = train)

Residuals:
     Min       1Q   Median       3Q      Max 
-14.4641  -1.1324   0.0089   1.1756  11.6171 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -4.162e+02  1.322e+01 -31.486  < 2e-16 ***
lat         -3.613e-02  9.212e-04 -39.221  < 2e-16 ***
long         2.978e-03  7.022e-04   4.240 2.24e-05 ***
month        1.133e-01  3.254e-03  34.818  < 2e-16 ***
year         2.446e-01  6.629e-03  36.898  < 2e-16 ***
cloudhigh    2.854e-02  1.320e-03  21.617  < 2e-16 ***
cloudlow    -8.341e-02  1.081e-03 -77.187  < 2e-16 ***
cloudmid     5.904e-03  1.696e-03   3.482 0.000498 ***
ozone       -2.693e-02  8.153e-04 -33.026  < 2e-16 ***
pressure     3.386e-02  3.129e-04 108.225  < 2e-16 ***
surftemp     6.783e-01  3.011e-03 225.229  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.897 on 28947 degrees of freedom
  (72 observations deleted due to missingne

In [14]:
pred2 <- predict(mod2,newdata=test)
rmse(test$temperature,pred2)
rmsle(test$temperature,pred2)
err_res <- rbind(err_res, data.frame(Name="Extended Linear regression", Model="mod2", 
                                     RMSE=rmse(test$temperature,pred2), 
                                     RMSLE=rmsle(test$temperature,pred2)))

[1] 1.876241

[1] 0.006330632

## Desicion trees

In [15]:
library(tree)
library(rpart)

In [16]:
mod3 <- tree(temperature ~., data=train)
mod3

node), split, n, deviance, yval
      * denotes terminal node

 1) root 28958 617400 298.0  
   2) surftemp < 292.95 5469 143300 291.8  
     4) surftemp < 286.55 937  19400 284.4 *
     5) surftemp > 286.55 4532  61550 293.4  
      10) lat < 27.4652 3406  39480 294.3  
        20) pressure < 817.5 204   3394 286.8 *
        21) pressure > 817.5 3202  24050 294.7 *
      11) lat > 27.4652 1126  10830 290.6 *
   3) surftemp > 292.95 23489 219800 299.4  
     6) surftemp < 297.6 11384 105900 297.8  
      12) cloudlow < 19.25 4348  36670 299.9  
        24) pressure < 947.5 367   6588 294.3 *
        25) pressure > 947.5 3981  17410 300.4 *
      13) cloudlow > 19.25 7036  38760 296.5  
        26) lat < 24.9696 5601  21080 297.1 *
        27) lat > 24.9696 1435   9735 294.4 *
     7) surftemp > 297.6 12105  59280 300.9  
      14) surftemp < 300.3 7100  31020 300.2 *
      15) surftemp > 300.3 5005  19890 301.9 *

In [17]:
pred3 <- predict(mod3,newdata=test)
rmse(test$temperature,pred3)
rmsle(test$temperature,pred3)
err_res <- rbind(err_res, data.frame(Name="Decision Trees-tree", Model="mod3", 
                                     RMSE=rmse(test$temperature,pred3), 
                                     RMSLE=rmsle(test$temperature,pred3)))

[1] 2.394362

[1] 0.008100497

In [18]:
mod4 <- rpart(temperature ~., data=train)
mod4

n= 29030 

node), split, n, deviance, yval
      * denotes terminal node

 1) root 29030 641599.600 297.9206  
   2) surftemp< 292.95 5540 154005.200 291.6720  
     4) surftemp< 286.55 970  20692.000 284.2220 *
     5) surftemp>=286.55 4570  68047.430 293.2533  
      10) pressure< 827.5 305   7149.498 286.3538 *
      11) pressure>=827.5 4265  45340.460 293.7467  
        22) lat>=27.46522 1093   9031.947 290.7864 *
        23) lat< 27.46522 3172  23429.110 294.7668 *
   3) surftemp>=292.95 23490 220273.600 299.3943  
     6) surftemp< 297.6 11385 106333.200 297.8214  
      12) cloudlow>=19.25 7037  39111.190 296.5338  
        24) lat>=24.96957 1435   9734.760 294.4374 *
        25) lat< 24.96957 5602  21454.360 297.0708 *
      13) cloudlow< 19.25 4348  36673.800 299.9053  
        26) pressure< 947.5 367   6588.420 294.2812 *
        27) pressure>=947.5 3981  17406.870 300.4238 *
     7) surftemp>=297.6 12105  59284.610 300.8736  
      14) surftemp< 300.3 7100  31017.560 300.174

In [19]:
pred4 <- predict(mod4,newdata=test)
rmse(test$temperature,pred4)
rmsle(test$temperature,pred4)
err_res <- rbind(err_res, data.frame(Name="Decision Trees-rpart", Model="mod4", 
                                     RMSE=rmse(test$temperature,pred4), 
                                     RMSLE=rmsle(test$temperature,pred4)))

[1] 2.388928

[1] 0.008080671

## Random Forest

In [20]:
#they are equivalent
library(randomForest) #not recommended for big data sets
library(ranger)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance




In [21]:
mod6 <- ranger(temperature ~., data=train)
mod6 #drop the missing

ERROR: Error: Missing data in columns: cloudlow.


In [ ]:
pred6 <- predict(mod6,data=test)
#head(pred6)
rmse(test$temperature,pred6$predictions)
rmsle(test$temperature,pred6$predictions)
err_res <- rbind(err_res, data.frame(Name="RandomForest (ranger)", Model="mod6", 
                                     RMSE=rmse(test$temperature,pred6$predictions), 
                                     RMSLE=rmsle(test$temperature,pred6$predictions)))